In [1]:
import pandas as pd
import numpy as np

import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import math
from collections import Counter
import re
import json
from bs4 import BeautifulSoup
import datetime
from pprint import pprint
import requests

In [2]:
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16

import keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Activation

Using TensorFlow backend.


# Collect all testing images

In [11]:
# get CIFAR10
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [40]:
# facial data
celeb_align = glob.glob('MsCelebV1-Faces-Aligned.Samples/MsCelebV1-Faces-Aligned.Samples/**/*.jpg')
celeb_crop = glob.glob('MsCelebV1-Faces-Cropped.Samples/MsCelebV1-Faces-Cropped.Samples/**/*.jpg')
celeb_full = glob.glob('MsCelebV1-ImageThumbnails.Samples/MsCelebV1-ImageThumbnails.Samples/**/*.jpg')
face = glob.glob('faces/*jpg')
lfw = glob.glob('lfw/**/A*.jpg') # Only pcik images with 'A' as first letter, 1054 images

test_face = [celeb_align, celeb_crop, celeb_full, face, lfw]

# non-facial data
background = glob.glob('cars_brad_bg/*.jpg')
car_rear = glob.glob('cars_markus/*jpg')
landscape = glob.glob('val_256/*.jpg')

test_nonface = [background, car_rear, landscape[0:1500]]

### Preprocess images

In [7]:
# convert to array and resize it from the path list
def image_process(path_list, size):
    test_group =[]
    for i in range(0, len(path_list)): 
        temp_img = cv2.imread(path_list[i])
        temp_img_resize = cv2.resize(temp_img, size)
        test_group.append(temp_img_resize)        
    return test_group

In [41]:
test_face_re = list(map(lambda t: image_process(t, (32, 32)), test_face))

In [31]:
test_nonface_re = list(map(lambda t: image_process(t, (32, 32)), test_nonface))

# add CIFAR10 data to non-facial images
test_nonface_re.append(x_train[2000:3500])

# model

In [9]:
# Reload structure
from pathlib import Path
f = Path('model_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model_weights.h5")

In [42]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9825454545454545
0.03636363636363636
0.027888446215139442
0.02
0.013282732447817837


In [43]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.00291970802919708
0.0
0.002
0.0


# model_slice

In [44]:
# Reload structure
from pathlib import Path
f = Path('model_structure_slice.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model_weights_slice.h5")

In [45]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9970909090909091
0.376
0.21779548472775564
0.31333333333333335
0.5673624288425048


In [46]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.10218978102189781
0.007936507936507936
0.05466666666666667
0.012


# model3

In [111]:
# convert to array and resize it from the path list
def image_process_gray(path_list, size):
    test_group =[]
    for i in range(0, len(path_list)): 
        temp_img = cv2.imread(path_list[i])
        temp_img_gray = cv2.cvtColor(temp_img, cv2.COLOR_BGR2GRAY)
        temp_img_gray_re = cv2.resize(temp_img_gray, size)
        temp_img_gray_re = temp_img_gray_re.reshape(32, 32, 1) # add one more layer
        test_group.append(temp_img_gray_re)        
    return test_group

In [114]:
test_face_gray_re = list(map(lambda t: image_process_gray(t, (32, 32)), test_face))

In [112]:
test_nonface_gray_re = list(map(lambda t: image_process_gray(t, (32, 32)), test_nonface))

In [113]:
np.array(test_nonface_gray_re[0]).shape

(1370, 32, 32, 1)

In [100]:
# Reload structure
from pathlib import Path
f = Path('model3_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model3_weights.h5")

In [115]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_gray_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9941818181818182
0.16654545454545455
0.10557768924302789
0.08666666666666667
0.3605313092979127


In [116]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_gray_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.03941605839416058
0.0
0.016666666666666666


# model4

In [47]:
# Reload structure
from pathlib import Path
f = Path('model4_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model4_weights.h5")

In [48]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9905454545454545
0.08436363636363636
0.05976095617529881
0.035555555555555556
0.23244781783681215


In [49]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.00583941605839416
0.0
0.0026666666666666666
0.009333333333333334


# model5

In [50]:
# Reload structure
from pathlib import Path
f = Path('model5_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model5_weights.h5")

In [51]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.984
0.9716363636363636
0.8857901726427623
0.7466666666666667
0.8624288425047438


In [52]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0
0.0
0.026
0.064


# model6

In [53]:
# Reload structure
from pathlib import Path
f = Path('model6_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model6_weights.h5")

In [54]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.31636363636363635
0.4741818181818182
0.14873837981407703
0.5088888888888888
1.0


In [55]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0
0.0
0.004
0.005333333333333333


# model7

In [57]:
# Reload structure
from pathlib import Path
f = Path('model6_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model7_weights.h5")

In [58]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.2829090909090909
0.5752727272727273
0.23041168658698538
0.5288888888888889
0.9990512333965844


In [59]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0021897810218978104
0.0
0.011333333333333334
0.012


# model8

In [60]:
# Reload structure
from pathlib import Path
f = Path('model6_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model8_weights.h5")

In [61]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9905454545454545
0.9563636363636364
0.5225763612217795
0.5133333333333333
0.9117647058823529


In [62]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.00145985401459854
0.0
0.008666666666666666
0.04533333333333334


# model3_aug (trained on 32 x 32)

In [73]:
# Reload structure
from pathlib import Path
f = Path('model3_aug_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model3_aug_weights.h5")

In [74]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.978909090909091
0.48363636363636364
0.43227091633466136
0.5733333333333334
0.7666034155597723


In [75]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.029197080291970802
0.15873015873015872
0.424
0.6


# model2_aug (trained on 64 x 64)

In [76]:
test_face_re_64 = list(map(lambda t: image_process(t, (64, 64)), test_face))

In [77]:
test_nonface_re_64 = list(map(lambda t: image_process(t, (64, 64)), test_nonface))

In [78]:
# Reload structure
from pathlib import Path
f = Path('model2_aug_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model2_aug_weights.h5")

In [80]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re_64:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9876363636363636
0.6509090909090909
0.550464807436919
0.4266666666666667
0.8301707779886148


In [81]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re_64:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0036496350364963502
0.023809523809523808
0.41733333333333333


# model1_aug (trained on 150 x150)

In [82]:
test_face_re_150 = list(map(lambda t: image_process(t, (150,150)), test_face))
test_nonface_re_150 = list(map(lambda t: image_process(t, (150,150)), test_nonface))

In [83]:
# Reload structure
from pathlib import Path
f = Path('model1_aug_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model1_aug_weights.h5")

In [84]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re_150:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9629090909090909
0.7469090909090909
0.6653386454183267
0.48
0.8510436432637571


In [85]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re_150:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.012408759124087591
0.031746031746031744
0.5613333333333334


# model4_aug

In [86]:
# Reload structure
from pathlib import Path
f = Path('model4_aug_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model4_aug_weights.h5")

In [87]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.9963636363636363
0.9825454545454545
0.6135458167330677
0.6355555555555555
0.9535104364326376


In [89]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0, 1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0
0.0
0.010666666666666666
0.10466666666666667


In [94]:
# Reload structure
from pathlib import Path
f = Path('model5_aug_structure.json')
model_structure = f.read_text()

from keras.models import model_from_json
model = model_from_json(model_structure)

# Reload weights
model.load_weights("model5_aug_weights.h5")

In [95]:
from sklearn.metrics import accuracy_score

print ('Test facial images')
for i in test_face_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0 ,1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test facial images
0.7970909090909091
0.6494545454545455
0.38711819389110225
0.4444444444444444
0.5616698292220114


In [96]:
from sklearn.metrics import accuracy_score

print ('Test non-facial images')
for i in test_nonface_re:
    process = np.array(i).astype('float32')
    process /= 255
    # prediction
    result = model.predict(process)
    observe = np.array([[0.0, 1.0]] * len(process))
    score = accuracy_score(np.rint(result), observe)
    print(score)

Test non-facial images
0.0
0.007936507936507936
0.0033333333333333335
0.030666666666666665
